# OAO Reader - "OnAndOn" format of the Motion GPS

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class OaoReader(BaseReader):
    '''OAO file - "OnAndOn" format of the Motion GPS'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            
            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readData()


    def readData(self):
        '''Read GNSS frames into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            mode, payload = self.readFrame()
            
            if mode in (0x0ad4, 0x0ad5):
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanBuffer += payload
                self.numRecords += 1
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        dtype = self.getDatatype()
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def readFrame(self):
        '''Read OAO frame from buffer'''

        offset = self.bufferPtr

        mode = self.readUnsigned16LE()
        expectedChecksum = self.readUnsigned16LE()

        if mode == 0x0ad0:
            length = 512
        elif mode == 0x0ad1:
            length = 12
        elif mode == 0x0ad2 or mode == 0x0ad3:
            length = 34
        elif mode == 0x0ad4 or mode == 0x0ad5:
            length = 52
        else:
            raise ValueError('Unexpected mode 0x{:x} - offset 0x{:08x}'.format(mode, offset))

        # Main frame content
        payload = self.readBytes(length - 4)

        # Evaluate checksum
        actualChecksum = self.calculateChecksum(mode, payload)
        if actualChecksum != expectedChecksum:
            raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        return mode, payload


    def calculateChecksum(self, mode, payload):
        '''Calculate Fletcher's checksum for the OAO frame'''

        checksum_a = checksum_b = 0
        
        # Sum the two first bytes
        for i in range(2):
            checksum_a += mode
            checksum_b += checksum_a
            mode >>= 8

        # Sum the payload bytes
        for byte in payload:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self):
        '''Get the datatype of the GNSS frames'''

        # Standard 32-byte GNSS frame - 48 bytes excluding the mode and checksum
        dtype = [
            ('latitude', '<i4'),
            ('longitude', '<i4'),
            ('altitude_msl', '<i4'),
            ('speed', '<u4'),
            ('course', '<u4'),
            ('utc_time', '<u8'),
            ('fix', 'u1'),
            ('satellites', 'u1'),
            ('speed_accuracy', '<u4'),
            ('horizontal_accuracy', '<u4'),
            ('vertical_accuracy', '<u4'),
            ('heading_accuracy', '<u4'),
            ('hdop', '<u2')
        ]

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['latitude'] / 10000000
        self.data['lon'] = self.rawData['longitude'] / 10000000

        # Convert elevation from cm to m
        self.data['ele'] = self.rawData['altitude_msl'] / 1000

        # Convert SOG from mm/s to m/s
        self.data['sog'] = self.rawData['speed'] / 1000

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['course'] / 100000

        # Convert timestamp from ms to s
        self.data['timestamp'] = self.rawData['utc_time'] / 1000

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fix']
        self.data['sat'] = self.rawData['satellites']

        # Convert sAcc from mm/s to m/s
        self.data['sacc'] = self.rawData['speed_accuracy'] / 1000

        # Convert hAcc and vAcc from mm to m
        self.data['hacc'] = self.rawData['horizontal_accuracy'] / 1000
        self.data['vacc'] = self.rawData['vertical_accuracy'] / 1000

        # Convert cAcc to degrees
        self.data['cacc'] = self.rawData['heading_accuracy'] / 100000

        # Convert HDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 100

## Unit Tests

In [3]:
class TestMiniData(unittest.TestCase):
    '''Class to test Motion Mini data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(miniReader.numRecords, 47980)

        for fieldName in miniReader.data:
            self.assertEqual(miniReader.data[fieldName].size, 47980)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(miniReader.data['hdop'].min(), 0.62)
        self.assertEqual(miniReader.data['hdop'].max(), 99.99)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(miniReader.data['sat'].min(), 0)
        self.assertEqual(miniReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(miniReader.data['timestamp'].min(), 1649672228.0)
        self.assertEqual(miniReader.data['timestamp'].max(), 1649677025.9)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(miniReader.data['lat'].min(), 50.5710305)
        self.assertEqual(miniReader.data['lat'].max(), 50.5833157)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(miniReader.data['lon'].min(), -2.462027)
        self.assertEqual(miniReader.data['lon'].max(), -2.4563075)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(miniReader.data['ele'].min(), -3.169)
        self.assertEqual(miniReader.data['ele'].max(), 13.548)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(miniReader.data['sog'].min(), 0)
        self.assertEqual(miniReader.data['sog'].max(), 17.142)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(miniReader.data['cog'].min(), 0.00056)
        self.assertEqual(miniReader.data['cog'].max(), 359.99618)


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(miniReader.data['fix'].min(), 0)
        self.assertEqual(miniReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(miniReader.data['sacc'].min(), 0.092)
        self.assertEqual(miniReader.data['sacc'].max(), 2.118)


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(miniReader.data['hacc'].min(), 0.395)
        self.assertEqual(miniReader.data['hacc'].max(), 12.051)


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(miniReader.data['vacc'].min(), 0.524)
        self.assertEqual(miniReader.data['vacc'].max(), 11.574)


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(miniReader.data['cacc'].min(), 0.40106)
        self.assertEqual(miniReader.data['cacc'].max(), 45.01615)

In [4]:
class TestEspData(unittest.TestCase):
    '''Class to test ESP-GPS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(espReader.numRecords, 43444)

        for fieldName in espReader.data:
            self.assertEqual(espReader.data[fieldName].size, 43444)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        # Note: HDOP in this file contains NAV-PVT.pDOP instead of NAV-DOP.hDOP
        self.assertEqual(espReader.data['hdop'].min(), 0.96)
        self.assertEqual(espReader.data['hdop'].max(), 1.88)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(espReader.data['sat'].min(), 9)
        self.assertEqual(espReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(espReader.data['timestamp'].min(), 1640864825.3)
        self.assertEqual(espReader.data['timestamp'].max(), 1640869169.6)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(espReader.data['lat'].min(), 51.6993591)
        self.assertEqual(espReader.data['lat'].max(), 51.7087559)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(espReader.data['lon'].min(), 4.0860906)
        self.assertEqual(espReader.data['lon'].max(), 4.0890658)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(espReader.data['ele'].min(), -3.159)
        self.assertEqual(espReader.data['ele'].max(), 11.978)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(espReader.data['sog'].min(), 0)
        self.assertEqual(espReader.data['sog'].max(), 18.169)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(espReader.data['cog'].min(), 0.01099)
        self.assertEqual(espReader.data['cog'].max(), 359.99525)


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(espReader.data['fix'].min(), 3)
        self.assertEqual(espReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(espReader.data['sacc'].min(), 0.051)
        self.assertEqual(espReader.data['sacc'].max(), 0.771)


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(espReader.data['hacc'].min(), 0.169)
        self.assertEqual(espReader.data['hacc'].max(), 2.508)


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(espReader.data['vacc'].min(), 0.207)
        self.assertEqual(espReader.data['vacc'].max(), 2.582)


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(espReader.data['cacc'].min(), 0.23885)
        self.assertEqual(espReader.data['cacc'].max(), 44.99855)

In [5]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
    miniReader = OaoReader(filename)

    filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
    espReader = OaoReader(filename)

    pc1 = time.perf_counter()
    miniReader.load()
    espReader.load()
    pc2 = time.perf_counter()

    print("\nOAO files loaded in in %0.2f seconds" % (pc2 - pc1))


OAO files loaded in in 0.50 seconds


In [6]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

............................
----------------------------------------------------------------------
Ran 28 tests in 0.019s

OK
